In [2]:
require 'image';

In [38]:
function showLCN(f,factor)
    local orig = image.load(f)
    orig = image.scale(orig, 400*factor,300*factor)
    local yuv = image.rgb2yuv(orig)
    local grey = yuv[1]
    print(torch.type(grey))
    local lcn = image.lcn(grey,image.gaussian({size=21, sigma=1.591/9,normalize=true}))
    --itorch.image(lcn)
    local max = lcn:max()
    local min = lcn:min()
    local H,W = grey:size(1),grey:size(2)
    local d = 10
    lcn = (lcn-min)/(max-min)
    grey:fill(0.5)
    grey[{{d+1,H-d},{d+1,W-d}}] = lcn
    return grey
end

In [45]:
function lcnscale(f,factor)
    local orig = image.load(f)
    orig = image.scale(orig, 400*factor,300*factor)
    local yuv = image.rgb2yuv(orig)
    local grey = yuv[1]
    return grey
end

In [49]:
function lcnThresh(f,th)
    local th = th or 0.5
    local img = image.load(f)
    img[img:gt(0.5)] = 1
    img[img:lt(0.5)] = 0
    return img
end

In [54]:
image.save('lcnttest1.png',lcnThresh('lcntest1.png'))